In [1]:
'''
5 baseline models:
    Random Forest
    Extra Trees
    Gradient Boost
    Ada Boost
    SVC
1 stack model:
    XGBoost
    
Pretty sure we dont need to get_dummies or scale,
will try and see how that goes and implemnet if needed'''

'\n5 baseline models:\n    Random Forest\n    Extra Trees\n    Gradient Boost\n    Ada Boost\n    SVC\n1 stack model:\n    XGBoost\n    \nPretty sure we dont need to get_dummies or scale,\nwill try and see how that goes and implemnet if needed'

In [4]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
sns.set()
from IPython.display import display

import random
import os
# print(os.listdir("../input"))

In [5]:
#Import modules specific to the machine learning aspect of this analysis#Import 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import Imputer

#Import classification algorithm modules
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from xgboost import plot_importance as x_plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import KFold

In [6]:
#Import scoring modules
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, average_precision_score

In [3]:
'''
#Uncomment this for kaggle
POS_CASH_balance = ('../input/POS_CASH_balance.csv')
test = pd.read_csv('../input/application_test.csv')
train = pd.read_csv('../input/application_train.csv')
bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
installments_payments = pd.read_csv('../input/installments_payments.csv')
previous_application = pd.read_csv('../input/previous_application.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')

'''
#Uncomment this for local
POS_CASH_balance = pd.read_csv('data/POS_CASH_balance.csv')
test = pd.read_csv('data/application_test.csv')
train = pd.read_csv('data/application_train.csv')
bureau = pd.read_csv('data/bureau.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
installments_payments = pd.read_csv('data/installments_payments.csv')
previous_application = pd.read_csv('data/previous_application.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

NameError: name 'pd' is not defined

In [6]:
train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [7]:
zero_train = train[train['TARGET']==0]
one_train = train[train['TARGET']==1]
zero_train['TARGET'].value_counts()

0    282686
Name: TARGET, dtype: int64

In [8]:
new_train = pd.concat([zero_train.head(24825), one_train], axis = 0)

In [9]:
new_train['TARGET'].value_counts()

1    24825
0    24825
Name: TARGET, dtype: int64

In [10]:
#highly imbalanced classes
#lets merge our train and test to create a bigger dataset to preprocess on
test['is_test'] = 1 
test['is_train'] = 0
new_train['is_test'] = 0
new_train['is_train'] = 1

In [11]:
X_train = new_train.drop(['TARGET'], axis = 1)
y_train = new_train['TARGET']
X_train.shape

(49650, 123)

In [12]:
X_test = test
X_test.shape

(48744, 123)

In [13]:
data = pd.concat([X_train, X_test], axis=0)

In [14]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [15]:
missing_values_table(data)

Your selected dataframe has 123 columns.
There are 66 columns that have missing values.


,Missing Values,% of Total Values
COMMONAREA_MODE,69217,70.3
COMMONAREA_MEDI,69217,70.3
COMMONAREA_AVG,69217,70.3
NONLIVINGAPARTMENTS_MEDI,68837,70.0
NONLIVINGAPARTMENTS_AVG,68837,70.0
NONLIVINGAPARTMENTS_MODE,68837,70.0
FONDKAPREMONT_MODE,67832,68.9
LIVINGAPARTMENTS_MODE,67811,68.9
LIVINGAPARTMENTS_MEDI,67811,68.9
LIVINGAPARTMENTS_AVG,67811,68.9


In [16]:
# Get the columns with > 60% missing
missing_df = missing_values_table(data);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 60].index)
print('We will remove %d columns.' % len(missing_columns))

Your selected dataframe has 123 columns.
There are 66 columns that have missing values.
We will remove 17 columns.


In [17]:
# Drop the columns
data = data.drop(labels = list(missing_columns), axis=1)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98394 entries, 1 to 48743
Columns: 106 entries, SK_ID_CURR to is_train
dtypes: float64(49), int64(42), object(15)
memory usage: 80.3+ MB


In [19]:
missing_data = data[data.columns[data.isnull().sum()>0]]
missing_data.dtypes

AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
OCCUPATION_TYPE                  object
EXT_SOURCE_1                    float64
EXT_SOURCE_2                    float64
EXT_SOURCE_3                    float64
APARTMENTS_AVG                  float64
BASEMENTAREA_AVG                float64
YEARS_BEGINEXPLUATATION_AVG     float64
ELEVATORS_AVG                   float64
ENTRANCES_AVG                   float64
FLOORSMAX_AVG                   float64
LANDAREA_AVG                    float64
LIVINGAREA_AVG                  float64
NONLIVINGAREA_AVG               float64
APARTMENTS_MODE                 float64
BASEMENTAREA_MODE               float64
YEARS_BEGINEXPLUATATION_MODE    float64
ELEVATORS_MODE                  float64
ENTRANCES_MODE                  float64
FLOORSMAX_MODE                  float64
LANDAREA_MODE                   float64
LIVINGAREA_MODE                 float64
NONLIVINGAREA_MODE              float64


In [20]:
for col in data.columns:
    if col in missing_data.columns and data[col].dtype == object:
        data[col].fillna('None', inplace=True)

In [21]:
missing_data = data[data.columns[data.isnull().sum()>0]]
missing_data.dtypes

AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
EXT_SOURCE_1                    float64
EXT_SOURCE_2                    float64
EXT_SOURCE_3                    float64
APARTMENTS_AVG                  float64
BASEMENTAREA_AVG                float64
YEARS_BEGINEXPLUATATION_AVG     float64
ELEVATORS_AVG                   float64
ENTRANCES_AVG                   float64
FLOORSMAX_AVG                   float64
LANDAREA_AVG                    float64
LIVINGAREA_AVG                  float64
NONLIVINGAREA_AVG               float64
APARTMENTS_MODE                 float64
BASEMENTAREA_MODE               float64
YEARS_BEGINEXPLUATATION_MODE    float64
ELEVATORS_MODE                  float64
ENTRANCES_MODE                  float64
FLOORSMAX_MODE                  float64
LANDAREA_MODE                   float64
LIVINGAREA_MODE                 float64
NONLIVINGAREA_MODE              float64
APARTMENTS_MEDI                 float64
BASEMENTAREA_MEDI               float64


In [22]:
data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_test,is_train
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,100008,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,...,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0,1


In [23]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']

for f_ in categorical_feats:
    data[f_] = pd.factorize(data[f_])[0]

In [24]:
data_cols = data.columns
imputer = Imputer(strategy='median')
imputer.fit(data)
data = imputer.transform(data)
data = pd.DataFrame(data=data, columns = data_cols, dtype='float64')
data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_test,is_train
0,100003.0,0.0,0.0,0.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100004.0,1.0,1.0,1.0,1.0,0.0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100006.0,0.0,0.0,0.0,1.0,0.0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
3,100007.0,0.0,1.0,0.0,1.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,100008.0,0.0,1.0,0.0,1.0,0.0,99000.0,490495.5,27517.5,454500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [25]:
ignore_features = ['is_train', 'is_test']
relevant_features = [col for col in data.columns if col not in ignore_features]
X_train = data[data['is_train']==1][relevant_features]
X_test = data[data['is_test']==1][relevant_features]
X_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100003.0,0.0,0.0,0.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100004.0,1.0,1.0,1.0,1.0,0.0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100006.0,0.0,0.0,0.0,1.0,0.0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100007.0,0.0,1.0,0.0,1.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100008.0,0.0,1.0,0.0,1.0,0.0,99000.0,490495.5,27517.5,454500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [26]:
X_train.shape

(49650, 104)

In [27]:
X_test.shape

(48744, 104)

In [28]:
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 38
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

class SklearnHelper(object):
    def __init__(self,clf,seed=0,params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self,x_train,y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self,x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [29]:
def get_oof(clf,x_train,y_train,x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS,ntest))
    
    for i, (train_index,test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        
        clf.train(x_tr,y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i,:] = clf.predict(x_test)
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [30]:
rf_params = {
    'n_jobs': -1
    , 'n_estimators': 10
    , 'max_depth': 8
    , 'min_samples_leaf': 2
    , 'verbose': 0
}

et_params = {
    'n_jobs': -1
    , 'n_estimators': 50
    , 'max_depth': 8
    , 'min_samples_leaf': 2
    , 'verbose': 0
}

ada_params = {
    'n_estimators': 50
    , 'learning_rate': .75
}

gb_params = {
    'n_estimators': 50
    , 'max_depth': 6
    , 'min_samples_leaf': 2
    , 'verbose': 0
}

svc_params = {
    'kernel': 'linear'
    , 'C': 0.025
}

In [31]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [32]:
y_train = y_train.ravel()
X_train = X_train.values
X_test = X_test.values

In [33]:
import timeit

In [34]:
start_time = timeit.default_timer()
et_oof_train, et_oof_test = get_oof(et, X_train, y_train, X_test) # Extra Trees
elapsed = timeit.default_timer() - start_time
elapsed , elapsed / 60

(3.727393868000945, 0.06212323113334908)

In [35]:
start_time = timeit.default_timer()
rf_oof_train, rf_oof_test = get_oof(rf,X_train, y_train, X_test) # Random Forest
elapsed = timeit.default_timer() - start_time
elapsed , elapsed / 60

(2.533912746992428, 0.042231879116540465)

In [36]:
start_time = timeit.default_timer()
ada_oof_train, ada_oof_test = get_oof(ada, X_train, y_train, X_test) # AdaBoost
elapsed = timeit.default_timer() - start_time
elapsed , elapsed / 60

(35.674235208993196, 0.5945705868165533)

In [ ]:
start_time = timeit.default_timer()
gb_oof_train, gb_oof_test = get_oof(gb,X_train, y_train, X_test) # Gradient Boost
elapsed = timeit.default_timer() - start_time
elapsed , elapsed / 60

(110.07509800299886, 1.8345849667166476)

In [ ]:
start_time = timeit.default_timer()
svc_oof_train, svc_oof_test = get_oof(svc,X_train, y_train, X_test) # Support Vector Classifier
elapsed = timeit.default_timer() - start_time
elapsed , elapsed / 60

In [ ]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

In [ ]:
model = XGBClassifier(
    max_depth = 7
    , n_estimators = 50
    , subsample = .9
    , colsample_bytree = .8
    , reg_alpha = .1
    , reg_lambda = .1
    , gamma = .9
    , objective = 'binary:logistic'
    , n_jobs = -1
)

In [ ]:
model.fit(x_train, y_train, eval_metric='auc')

In [ ]:
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype('int32')
test.head()

In [ ]:
test_preds = model.predict_proba(x_test)
pred_df = pd.DataFrame()
pred_df['SK_ID_CURR'] = test['SK_ID_CURR']
pred_df['TARGET'] = test_preds[:,1]
pred_df.to_csv('xgb_baseline.csv', index=False)
pred_df.head()

In [ ]:
'''next step considerations:
- feature selection (keep 95% variance) or maybe PCA???
- lightgbm
- k fold cross validation?
- merging tables

I also dont think that I have to normalize (minmaxscale) considering were using a tree method

some ideas:
its not quite clear what these two people are doing for null values in their code, other than that it has some ideas
https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code
https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-model/notebook'''